In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from google.colab import drive
from google.colab import runtime
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/AdaptOrDie/'
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/MyDrive/AdaptOrDie


# Pretraing Unipelt adapter-based model on TAPT dataset (test as train)

In [ ]:
!pip install datasets
!pip install transformers[torch]



!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none

In [ ]:
pretrain_train_dataset, pretrain_eval_dataset = load_dataset("imdb", split=["train","test"])

In [ ]:
print(pretrain_train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [ ]:
pretrain_train_dataset[0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
print(pretrain_train_dataset)

In [ ]:
from datasets import load_dataset

pretrain_dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=["test","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9754
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [1]:
# pretrain_train_dataset[0]

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from adapters import UniPELTConfig

config = UniPELTConfig()

pretraining_adapter_config = UniPELTConfig()
pretrained_adapter_name = "tapt_unipelt"
pretrained_adapter_hub_name = f"pretrain_{pretrained_adapter_name}_adpater"
print(pretraining_adapter_config.__class__.__name__)
print(pretrained_adapter_name)

UniPELTConfig
tapt_unipelt


In [ ]:
# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=pretraining_adapter_config)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")
print(pretrain_model.active_adapters)

Pretrained Adapter Name: tapt_unipelt
Stack[tapt_unipelt]


In [ ]:
summary = pretrain_model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
tapt_unipelt             union            11,083,376       8.892       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback


pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_unipelt_pretraining_model",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22*2*6 = 252
    num_train_epochs=100,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    torch_compile=False,  # True for base model, False for Adapter model
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True, #(edited)
    report_to="tensorboard",
)

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,4.784600,2.505015
2,2.142600,1.944875
4,1.950700,1.807567
6,1.859600,1.731062
8,1.802600,1.679249
10,1.766100,1.650391
12,1.733200,1.627951
14,1.713500,1.621275
16,1.697400,1.591787
18,1.679400,1.597785


Checkpoint destination directory tapt_helpfulness_unipelt_pretraining_model/checkpoint-232 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=9532, training_loss=1.7941592973268088, metrics={'train_runtime': 13450.3288, 'train_samples_per_second': 72.519, 'train_steps_per_second': 1.725, 'total_flos': 1.204051998463058e+17, 'train_loss': 1.7941592973268088, 'epoch': 41.0})


In [ ]:
pretrain_model.push_adapter_to_hub(
    pretrained_adapter_hub_name,
    pretrained_adapter_name,
    datasets_tag=pretrain_dataset_name,
    overwrite_adapter_card=True,
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ltuzova/pretrain_tapt_unipelt_adpater/commit/dbf3c7c4a51262b5c9d444aeafb27d6c923598dc', commit_message='Upload model', commit_description='', oid='dbf3c7c4a51262b5c9d444aeafb27d6c923598dc', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
pretrained_adapter_name

'tapt_unipelt'

In [ ]:
pretrain_trainer.push_to_hub('tapt_helpfulness_unipelt_pretraining_model')

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

events.out.tfevents.1713324358.0ba9c08adc61.2619.0:   0%|          | 0.00/23.0k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ltuzova/tapt_helpfulness_unipelt_pretraining_model/commit/843b0bbedb4c432d9650636535155042b954d2e4', commit_message='tapt_helpfulness_unipelt_pretraining_model', commit_description='', oid='843b0bbedb4c432d9650636535155042b954d2e4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("Disconnecting Session")
runtime.unassign()

# Repeat Unipelt TAP pre-training (full train set)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from datasets import load_dataset

pretrain_dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=["train","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 44876
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from adapters import UniPELTConfig

config = UniPELTConfig()

pretraining_adapter_config = UniPELTConfig()
pretrained_adapter_name = "tapt_unipelt"
pretrained_adapter_hub_name = f"pretrain_{pretrained_adapter_name}_adpater_fix_train"
print(pretraining_adapter_config.__class__.__name__)
print(pretrained_adapter_name)

UniPELTConfig
tapt_unipelt


In [ ]:
# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=pretraining_adapter_config)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")
print(pretrain_model.active_adapters)

Pretrained Adapter Name: tapt_unipelt
Stack[tapt_unipelt]


In [ ]:
summary = pretrain_model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
tapt_unipelt             union            11,083,376       8.892       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback


pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_unipelt_pretraining_model_fix_train",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22*2*6 = 252
    num_train_epochs=20, #100,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    torch_compile=False,  # True for base model, False for Adapter model
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True, #(edited)
    report_to="tensorboard",
)

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)

In [ ]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

In [ ]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,2.753600,1.816130
2,1.757800,1.613423
4,1.675500,1.566016
6,1.640200,1.523649
8,1.614500,1.497001
10,1.600400,1.511974
12,1.588800,1.497378
14,1.578400,1.476749
16,1.571700,1.477599
18,1.568100,1.475641


TrainOutput(global_step=21360, training_loss=1.6834042838450227, metrics={'train_runtime': 27797.6495, 'train_samples_per_second': 32.288, 'train_steps_per_second': 0.768, 'total_flos': 2.7009713385208013e+17, 'train_loss': 1.6834042838450227, 'epoch': 19.99})


In [ ]:
pretrained_adapter_hub_name

NameError: name 'pretrained_adapter_hub_name' is not defined

In [ ]:
pretrain_model.push_adapter_to_hub(
    pretrained_adapter_hub_name,
    pretrained_adapter_name,
    datasets_tag=pretrain_dataset_name,
    overwrite_adapter_card=True,
)

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ltuzova/pretrain_tapt_unipelt_adpater_fix_train/commit/49b2c16d5ff222399a615eb57deee740a6ad99fa', commit_message='Upload model', commit_description='', oid='49b2c16d5ff222399a615eb57deee740a6ad99fa', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
pretrain_trainer.push_to_hub('tapt_helpfulness_unipelt_pretraining_model_fix_train')

events.out.tfevents.1713571847.599815aa97cd.1640.0:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ltuzova/tapt_helpfulness_unipelt_pretraining_model_fix_train/commit/7eda2b9136aaa27756299b049f4b7cb9ac4e5e61', commit_message='tapt_helpfulness_unipelt_pretraining_model_fix_train', commit_description='', oid='7eda2b9136aaa27756299b049f4b7cb9ac4e5e61', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("Disconnecting Session")
runtime.unassign()

Disconnecting Session


# Pretrainin Seq_Bn Adapter-based model on TAPT dataset (full train set)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from datasets import load_dataset

pretrain_dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=["train","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 44876
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from adapters import SeqBnConfig

pretraining_adapter_config = SeqBnConfig()
pretrained_adapter_name = "tapt_seq_bn"
pretrained_adapter_hub_name = f"pretrain_{pretrained_adapter_name}_adpater_full_train"
print(pretraining_adapter_config.__class__.__name__)
print(pretrained_adapter_name)

SeqBnConfig
tapt_seq_bn


In [ ]:
# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=pretraining_adapter_config)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")
print(pretrain_model.active_adapters)

Pretrained Adapter Name: tapt_seq_bn
Stack[tapt_seq_bn]


In [ ]:
summary = pretrain_model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
tapt_seq_bn              bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback


pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_seq_bn_pretraining_model_full_train",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22*2*6 = 252
    num_train_epochs=20, #100,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    torch_compile=False,  # True for base model, False for Adapter model
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True, #(edited)
    report_to="tensorboard",
)

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)

In [ ]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,2.746000,1.830902
2,1.771100,1.627466
4,1.687600,1.578800
6,1.652600,1.534449
8,1.626000,1.509671
10,1.611000,1.524801
12,1.600300,1.507055
14,1.589300,1.489150
16,1.583900,1.489288
18,1.580800,1.487180


TrainOutput(global_step=21360, training_loss=1.6944962862279085, metrics={'train_runtime': 23035.2542, 'train_samples_per_second': 38.963, 'train_steps_per_second': 0.927, 'total_flos': 2.4203891127861043e+17, 'train_loss': 1.6944962862279085, 'epoch': 19.99})


In [ ]:
pretrain_model.push_adapter_to_hub(
    pretrained_adapter_hub_name,
    pretrained_adapter_name,
    datasets_tag=pretrain_dataset_name,
    overwrite_adapter_card=True,
)

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ltuzova/pretrain_tapt_seq_bn_adpater_full_train/commit/d56dd0aaf1b2aea225b739c450ddf812eb89d06f', commit_message='Upload model', commit_description='', oid='d56dd0aaf1b2aea225b739c450ddf812eb89d06f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
pretrain_trainer.push_to_hub('tapt_helpfulness_seq_bn_pretraining_model_full_train')

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

events.out.tfevents.1713600064.ea01de983f1c.331.0:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ltuzova/tapt_helpfulness_seq_bn_pretraining_model_full_train/commit/9a6e81b6fba121a6fbee1214c043fe71a628fa81', commit_message='tapt_helpfulness_seq_bn_pretraining_model_full_train', commit_description='', oid='9a6e81b6fba121a6fbee1214c043fe71a628fa81', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("Disconnecting Session")
runtime.unassign()

Disconnecting Session


# Pretrainin No adapter model on TAPT dataset (full train set)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from datasets import load_dataset

pretrain_dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=["train","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 44876
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [ ]:
from transformers import AutoConfig, AutoModelForMaskedLM
pretrain_model = AutoModelForMaskedLM.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.push_to_hub("tapt_helpfulness_base_pretraining_model_full_train")

CommitInfo(commit_url='https://huggingface.co/ltuzova/tapt_helpfulness_base_pretraining_model_full_train/commit/f2bc1f8f5d5035ec141978801836ca7e3f8f7dd5', commit_message='Upload tokenizer', commit_description='', oid='f2bc1f8f5d5035ec141978801836ca7e3f8f7dd5', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [ ]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_base_pretraining_model_full_train",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=20,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1, # Saves latest 2 checkpoints
    torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [ ]:
from transformers import EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [ ]:
from transformers import Trainer

pretrain_trainer = Trainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [ ]:
before_pretrain_eval_results = pretrain_trainer.evaluate()
before_pretrain_eval_results_loss = before_pretrain_eval_results["eval_loss"]
print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")
print(before_pretrain_eval_results)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Before pretraining loss: 1.9861
{'eval_loss': 1.9861068725585938, 'eval_runtime': 56.5961, 'eval_samples_per_second': 34.985, 'eval_steps_per_second': 1.679}


In [ ]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

Epoch,Training Loss,Validation Loss
0,1.686200,1.526152
2,1.542300,1.434600
4,1.479200,1.409991
6,1.435000,1.385083
8,1.396800,1.356703
10,1.366200,1.334734
12,1.338200,1.323235
14,1.313800,1.315704
16,1.291600,1.287102
18,1.273800,1.282500


There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


TrainOutput(global_step=21360, training_loss=1.399364960059691, metrics={'train_runtime': 11002.2015, 'train_samples_per_second': 81.576, 'train_steps_per_second': 1.941, 'total_flos': 2.361757103883049e+17, 'train_loss': 1.399364960059691, 'epoch': 19.99})


In [ ]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

After pretraining loss: 1.2718
{'eval_loss': 1.2717572450637817, 'eval_runtime': 10.4078, 'eval_samples_per_second': 190.242, 'eval_steps_per_second': 9.128, 'epoch': 19.99}


In [ ]:
pretrain_trainer.push_to_hub("tapt_helpfulness_base_pretraining_model_full_train")

events.out.tfevents.1713655022.cda4fd27c114.752.1:   0%|          | 0.00/364 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ltuzova/tapt_helpfulness_base_pretraining_model_full_train/commit/75daea20a290a14d361e97eba89a376a31f680d8', commit_message='tapt_helpfulness_base_pretraining_model_full_train', commit_description='', oid='75daea20a290a14d361e97eba89a376a31f680d8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("Disconnecting Session")
runtime.unassign()

Disconnecting Session


## DAPT/TAPT pretrain - upload token

In [ ]:
from datasets import load_dataset

pretrain_dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=["test","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9754
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [ ]:
from transformers import AutoConfig, AutoModelForMaskedLM
pretrain_model = AutoModelForMaskedLM.from_pretrained("BigTMiami/amazon_pretraining_5M_model_corrected")

In [ ]:
from transformers import AutoTokenizer
pretrain_model = AutoModelForMaskedLM.from_pretrained("BigTMiami/tapt_helpfulness_base_pretraining_model")
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
pretrain_model.push_to_hub("tapt_helpfulness_base_pretraining_model")
tokenizer.push_to_hub("tapt_helpfulness_base_pretraining_model")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ltuzova/tapt_helpfulness_base_pretraining_model/commit/ac03898477c05cf82de5dd6435132dd7386f60f1', commit_message='Upload tokenizer', commit_description='', oid='ac03898477c05cf82de5dd6435132dd7386f60f1', pr_url=None, pr_revision=None, pr_num=None)